## experimenting...

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate, populate

reset_and_populate(raw, processed, [300,100,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

## Data generator

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

# train and validation generator are virtually the same, but is still defined in good taste

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 10, train_datagen)
validation_generator = create_generator('validation', 10, validation_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


## Create Model

Below a model defining function is defined. 
The function creates a model with:
- n layers (between 2 and 5)
- 

In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, RMSprop

def create_model(hp):
    model = models.Sequential()
    # Invariant Input
    model.add(layers.Flatten(input_shape=(32, 32, 1)))
    # Add n layers
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(
            units=hp.Int('num_nodes', 16, 64, 4),
            activation='relu'))
    # Invariant Output
    model.add(layers.Dense(3, 'softmax'))
    # Select any Optimizer
    optimizers = {
        'sgd': SGD(
            lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01]),
            momentum=hp.Float('momentum', 0.5, 1, 0.1),
            nesterov=hp.Boolean('nesterov'))
    }
    # Compile Model
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', ['sgd'])],
        metrics=['acc'])

    return model

## Custom KerasTuner Class

Create a subclass to implement features for visualization of the 

In [4]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import RandomSearch
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(RandomSearch):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def run_trial(self, trial, *args, **kwargs):
        callbacks = kwargs.pop('callbacks', [])
        callbacks = self._deepcopy_callbacks(callbacks)
        trial_dir = self.get_trial_dir(trial.trial_id)
        callbacks.append(TensorBoard(trial_dir))
        kwargs['callbacks'] = callbacks
        super().run_trial(trial, *args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)
        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        print('Remaining Trials: ' + str(self.remaining_trials))
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [5]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = join('logs', 'srp13')
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_trials=500,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=30,
    validation_steps=30,
    verbose=0,
    callbacks=callbacks)

Remaining Trials: 499
Remaining Trials: 498
Remaining Trials: 497
Remaining Trials: 496
Remaining Trials: 495
Remaining Trials: 494
Remaining Trials: 493
Remaining Trials: 492
Remaining Trials: 491
Remaining Trials: 490
Remaining Trials: 489
Remaining Trials: 488
Remaining Trials: 487
Remaining Trials: 486
Remaining Trials: 485
Remaining Trials: 484
Remaining Trials: 483
Remaining Trials: 482
Remaining Trials: 481
Remaining Trials: 480
Remaining Trials: 479
Remaining Trials: 478
Remaining Trials: 477
Remaining Trials: 476
Remaining Trials: 475
Remaining Trials: 474
Remaining Trials: 473
Remaining Trials: 472
Remaining Trials: 471
Remaining Trials: 470
Remaining Trials: 469
Remaining Trials: 468
Remaining Trials: 467
Remaining Trials: 466
Remaining Trials: 465
Remaining Trials: 464
Remaining Trials: 463
Remaining Trials: 462
Remaining Trials: 461
Remaining Trials: 460
Remaining Trials: 459
Remaining Trials: 458
Remaining Trials: 457
Remaining Trials: 456
Remaining Trials: 455
Remaining 

In [0]:
tuner.results_summary()